# Live Model Server Testing

Test your model server via HTTP calls

```markdown
:param table:         csv/parquet table with test data
:param addr:          function address/url
:param label_column:  name of the label column in table
:param model:         tested model name 
:param match_err:     raise error on validation (require proper test set)
:param rows:          number of rows to use from test set

```

## Deploy model server for testing

In [1]:
import os
import mlrun

project_name = "sk-project"
MODEL_PATH = "https://s3.wasabisys.com/iguazio/models/iris/model.pkl"

_, artifact_path = mlrun.set_environment(
    api_path="http://mlrun-api:8080", artifact_path=os.path.abspath("./")
)

# import model server function from hub
fn = mlrun.import_function("hub://model_server")
fn.add_model("mymodel", MODEL_PATH)
address = fn.deploy()

> 2021-02-16 12:50:25,766 [info] Starting remote function deploy
2021-02-16 12:50:25  (info) Deploying function
2021-02-16 12:50:25  (info) Building
2021-02-16 12:50:25  (info) Staging files and preparing base images
2021-02-16 12:50:25  (info) Building processor image
2021-02-16 12:50:27  (info) Build complete
2021-02-16 12:50:35  (info) Function deploy complete
> 2021-02-16 12:50:36,977 [info] function deployed, address=default-tenant.app.vmdev36.lab.iguazeng.com:32699


In [2]:
user_name = os.getenv("V3IO_USERNAME")

fn.apply(mlrun.mount_v3io())
fn.set_envs(
    {
        "SERVING_MODEL_iris_dataset_v1": MODEL_PATH,
        "INFERENCE_STREAM": "users/{}/tststream".format(user_name),
    }
)

address = fn.deploy()

> 2021-02-16 12:50:36,988 [info] Starting remote function deploy
2021-02-16 12:50:37  (info) Deploying function
2021-02-16 12:50:37  (info) Building
2021-02-16 12:50:37  (info) Staging files and preparing base images
2021-02-16 12:50:37  (info) Building processor image
2021-02-16 12:50:38  (info) Build complete
2021-02-16 12:50:48  (info) Function deploy complete
> 2021-02-16 12:50:49,591 [info] function deployed, address=default-tenant.app.vmdev36.lab.iguazeng.com:32699


## Run model server tester locally

In [3]:
from model_server_tester import model_server_tester

# run the function locally
DATA_PATH = "https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv"

gen = mlrun.run_local(
    name="model_server_tester",
    handler=model_server_tester,
    params={"addr": address, "model": "mymodel"},
    inputs={"table": DATA_PATH},
    project=project_name,
    artifact_path=os.path.join(artifact_path, "data"),
)

> 2021-02-16 12:50:49,627 [info] starting run model_server_tester uid=aacf6b06d34540c49d3cd31f7b2e1496 DB=http://mlrun-api:8080
> 2021-02-16 12:50:50,704 [info] testing with dataset against http://default-tenant.app.vmdev36.lab.iguazeng.com:32699, model: mymodel
> 2021-02-16 12:50:51,940 [info] run 20 tests, 0 errors and 8 match expected value


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...7b2e1496,0,Feb 16 12:50:49,completed,model_server_tester,v3io_user=adminkind=handlerowner=adminhost=jupyter-7b854d9bd6-mkmbn,table,addr=http://default-tenant.app.vmdev36.lab.iguazeng.com:32699model=mymodel,total_tests=20errors=0match=8avg_latency=54462min_latency=39743max_latency=243248,latency


to track results use .show() or .logs() or in CLI: 
!mlrun get run aacf6b06d34540c49d3cd31f7b2e1496 --project sk-project , !mlrun logs aacf6b06d34540c49d3cd31f7b2e1496 --project sk-project
> 2021-02-16 12:50:52,075 [info] run executed, status=completed


## Save

In [4]:
import yaml

with open("item.yaml") as item_file:
    items = yaml.load(item_file, Loader=yaml.FullLoader)

In [5]:
test_func = mlrun.code_to_function(
    name=items["name"],
    kind=items["spec"]["kind"],
    filename=items["spec"]["filename"],
    image=items["spec"]["image"],
    description=items["description"],
    categories=items["categories"],
    labels=items["labels"],
)

test_func.export("model_server_tester.yaml")

> 2021-02-16 12:50:52,468 [info] function spec saved to path: model_server_tester.yaml


## Run remotely

In [6]:
test_func.run(
    mlrun.NewTask(
        name="model_server_tester",
        handler=model_server_tester,
        params={"addr": address, "model": "mymodel"},
        inputs={"table": DATA_PATH},
        project=project_name,
        artifact_path=os.path.join(artifact_path, "data"),
    )
)

> 2021-02-16 12:50:52,479 [info] starting run model_server_tester uid=a0e2796c9ec84753b8fffe4ab28659f4 DB=http://mlrun-api:8080
> 2021-02-16 12:50:52,708 [info] Job is running in the background, pod: model-server-tester-2vtb4
> 2021-02-16 12:50:56,295 [info] testing with dataset against http://default-tenant.app.vmdev36.lab.iguazeng.com:32699, model: mymodel
> 2021-02-16 12:50:57,198 [info] run 20 tests, 0 errors and 7 match expected value
> 2021-02-16 12:50:57,252 [info] run executed, status=completed

final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...b28659f4,0,Feb 16 12:50:55,completed,model_server_tester,v3io_user=adminkind=jobowner=adminhost=model-server-tester-2vtb4,table,addr=http://default-tenant.app.vmdev36.lab.iguazeng.com:32699model=mymodel,total_tests=20errors=0match=7avg_latency=40460min_latency=38337max_latency=45480,latency


to track results use .show() or .logs() or in CLI: 
!mlrun get run a0e2796c9ec84753b8fffe4ab28659f4 --project sk-project , !mlrun logs a0e2796c9ec84753b8fffe4ab28659f4 --project sk-project
> 2021-02-16 12:52:07,364 [info] run executed, status=completed
